In [3]:
import os
import absl
import urllib
import tempfile

import tensorflow as tf
import tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)

In [2]:
_data_root = tempfile.mkdtemp()
_data_filepath = os.path.join(_data_root, 'data.csv')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tmpiwr9hxpn/data.csv', <http.client.HTTPMessage at 0x7fc8e61c5790>)

In [4]:
context = InteractiveContext()

- Consumes: tf.Examples from an ExampleGen component, and a data schema from a SchemaGen component.
- Emits: A SavedModel to a Trainer component

When executed, the SavedModel will accept tf.Examples emitted from an ExampleGen component and emit the transformed feature data.

## Configuring a Transform Component

In [10]:
from tfx.utils.dsl_utils import external_input
from tfx.components import CsvExampleGen
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tmpiwr9hxpn/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/CsvExampleGen/examples/1, id: 2)]
        ))

In [11]:
from tfx.components import StatisticsGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/StatisticsGen/statistics/2/eval.
/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/StatisticsGen/statistics/2, id: 3)]
        ))

In [13]:
from tfx.components import SchemaGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:absl:Schema written to /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/SchemaGen/schema/3, id: 4)]
        ))

In [15]:
# First define the transform_module.py file, define `preprocessing_fn` in that file, 
# which will be used to construct preprocessing pipeline

from tfx.components import Transform

transform_training = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    # Exactly one of 'module_file' or 'preprocessing_fn' must be supplied.
#     preprocessing_fn=...,
    module_file=os.path.abspath('taxi_transform.py'), # path/to/module_file
    instance_name='transform-training'
)
context.run(transform_training)

INFO:absl:Running driver for Transform.transform-training
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform.transform-training


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:absl:Using 1 process(es) for Beam pipeline execution.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/Transform.transform-training/transform_graph/4/.temp_path/tftransform_tmp/4976665283ae448798853552a2d6e84f/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-16

INFO:absl:Running publisher for Transform.transform-training
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform.transform-training
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/Transform.transform-training/transform_graph/4, id: 5)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-16T00_35_42.399545-6qsw2l_b/Transform.transform-training/transformed_examples/4, id: 6)]
        ))

## Transform and TensorFlow Transform

Transform makes extensive use of TensorFlow Transform for performing feature engineering on your dataset. TensorFlow Transform is a great tool for transforming feature data before it goes to your model and as a part of the training process. Common feature transformations include:

- Embedding: converting sparse features (like the integer IDs produced by a vocabulary) into dense features by finding a meaningful mapping from high- dimensional space to low dimensional space. See the Embeddings unit in the Machine-learning Crash Course for an introduction to embeddings.
- Vocabulary generation: converting strings or other non-numeric features into integers by creating a vocabulary that maps each unique value to an ID number.
- Normalizing values: transforming numeric features so that they all fall within a similar range.
- Bucketization: converting continuous-valued features into categorical features by assigning values to discrete buckets.
- Enriching text features: producing features from raw data like tokens, n-grams, entities, sentiment, etc., to enrich the feature set.

## The TensorFlow Transform preprocessing_fn

The input to the preprocessing_fn is determined by the schema. A `Schema` proto contains a list of `Feature`s, and Transform converts these to a "feature spec" (sometimes called a "parsing spec") which is a dict whose keys are feature names and whose values are one of `FixedLenFeature` or `VarLenFeature` (or other options not used by TensorFlow Transform).

In [29]:
from typing import Dict, Text, Any
def preprocessing_fn(inputs: Dict[Text, Any]) -> Dict[Text, Any]:
    # where the values of input and returned Dict are either tf.Tensor or tf.SparseTensor.
    ...

- Each `feature` with `shape` set will result in a `tf.FixedLenFeature` with shape and `default_value=None`. `presence.min_fraction` must be `1` otherwise and error will result, since when there is no default value, a `tf.FixedLenFeature` requires the feature to always be present.
- Each `feature` with `shape` not set will result in a `VarLenFeature`.
- Each `sparse_feature` will result in a `tf.SparseFeature` whose `size` and `is_sorted` are determined by the `fixed_shape` and `is_sorted` fields of the `SparseFeature` message.
- Features used as the `index_feature` or `value_feature` of a sparse_feature will not have their own entry generated in the feature spec.
- The correspondence between `type` field of the `feature` (or the values feature of a `sparse_feature` proto) and the `dtype` of the feature spec is given by the following table:

| type | dtype |
|----|---|
|schema_pb2.INT|tf.int64|
|schema_pb2.FLOAT|tf.float32|
|schema_pb2.BYTES|tf.string|